[Data on Orgs not using year round giving](https://projects.zoho.com/portal/qgiv#buginfo/821217000000014017/821217000011971045)

I need a list of orgs that aren't using us for year round. Honestly - it would be nice to be able to get something like this on demand because I'd like to keep it updated HOWEVER, my needs are: 

### First

- client name
- ARR (trailing 12 starting January 2021) 
- CSM 
- NTEE 
- go live date 

I assume we'd figure this out by taking a look at those orgs that don't have a public form source during that timeframe?

### Second

​I also would like to get a baseline in how much we're processing in year round gifts (last year) - number of orgs and a revenue number. I don't know if we can do more than public form? Like mobile form could be P2P or Auction too right? So I think it would be based on public form only but interested in your ideas here. 

In [5]:
import pandas as pd
import numpy as np

import sys
sys.path.append("../../scripts/")
from s3_support import *

# 1. List of orgs not using year round

In [135]:
report_fields = ['Org Name', 'ARR', 'CSM', 'NTEE', 'Go Live Date']

In [169]:
q = '''select 
            org, source, count(id) as trans_count, sum(amount) as trans_sum
        from transactions_dup 
        where 
            date > dateadd(month, -12, CURRENT_DATE) and
            status='A'
        group by org, source'''
orgs = redshift_query_read(q, schema='public')

In [170]:
orgs.tail(3)

,org,source,trans_count,trans_sum
7592,44019,mobile,1,50.0
7593,441895,vt,1,250.0
7594,446110,don_form,1,1.0


In [171]:
trans_pvt = orgs.pivot(index='org', columns='source', values=['trans_count', 'trans_sum']).fillna(0)

In [172]:
for col in trans_pvt.columns:
    if col[0] != '' and col[1] != '':
        trans_pvt["{}_{}".format(col[1], col[0])] = trans_pvt[col]
    else:
        trans_pvt[col[0]] = trans_pvt[col]
    trans_pvt.drop(col, axis=1, inplace=True)

In [173]:
trans_pvt = trans_pvt.reset_index()
#trans_pvt.reset_index(level=[c[0] for c in trans_pvt.columns]).columns
trans_pvt.columns = trans_pvt.columns.get_level_values(0)
trans_pvt.tail(3)

,org,don_form_trans_count,fb_trans_count,givi_trans_count,kiosk_trans_count,mobile_trans_count,mobilevt_trans_count,p2p_trans_count,sms_trans_count,vt_trans_count,don_form_trans_sum,fb_trans_sum,givi_trans_sum,kiosk_trans_sum,mobile_trans_sum,mobilevt_trans_sum,p2p_trans_sum,sms_trans_sum,vt_trans_sum
3474,446691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3475,446700,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3476,446707,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [174]:
org_names = get_dataframe_from_file("qgiv-stats-data", "organizations.clean.csv")

In [175]:
org_df_cols = ['id', 'org_name', 'live_date', ]
orgs_df = trans_pvt.merge(org_names[org_df_cols], left_on='org', right_on='id').head()
orgs_df.drop(['id'], axis=1, inplace=True)

In [176]:
orgs_df.head(3)

,org,don_form_trans_count,fb_trans_count,givi_trans_count,kiosk_trans_count,mobile_trans_count,mobilevt_trans_count,p2p_trans_count,sms_trans_count,vt_trans_count,...,fb_trans_sum,givi_trans_sum,kiosk_trans_sum,mobile_trans_sum,mobilevt_trans_sum,p2p_trans_sum,sms_trans_sum,vt_trans_sum,org_name,live_date
0,6,612.0,0.0,0.0,0.0,290.0,123.0,97.0,0.0,308.0,...,0.0,0.00,0.0,26587.12,2675.76,5005.86,0.00,43986.88,Camp Fire Sunshine Central Florida,2006-05-18
1,13,696.0,0.0,71.0,0.0,31.0,0.0,139.0,17.0,81.0,...,0.0,6651.07,0.0,1694.71,0.00,11517.46,1045.36,23268.00,VISTE,2006-08-22
2,31,351.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.00,100.00,Ronald McDonald House of Eastern Iowa & Wester...,2007-05-18


__still need CSM and NTEE, not sure where to get those yet__

Production query to get transaction ARR per org:

Get org info:

```
SELECT
	ot.org,
    o.name,
    avg(ot.trans_revenue) as arr,
    o.csm as csm_id,
    a.username as csm,
    o.dateLive
FROM
    (SELECT 
        org, 
        sum(qgiv_fee) as trans_revenue, 
        year(tm_stamp) as year 
    from Transaction 
    where 
        year(tm_stamp) != 2021 AND
     	status='A'
    group by org, year(tm_stamp)) as ot
left join Organization as o
    on o.id=ot.org
left join GlobalAccounts as a
    on o.csm=a.id
group by org;
```

Get org's by year round transaction counts:

```
select 
	t.org,
	count(t.id) as trans_count
    year(curdate()) - year(o.dateLive) as years_live,
    count(t.id) / (year(curdate()) - year(o.dateLive)) as year_round_per_year
from Transaction as t
	left join Form as f on t.form=f.id
    left join Organization as o on o.id=t.org
where 
	(t.source='don_form' or t.source='mobile') and 
    t.tm_stamp>='2020-01-01' and 
    f.type=1
group by t.org
```

- this does not account for system fees (recurring and one time)
- still need NTEE, not sure where that is

# 2. Year round baseline processing 

product type:
1. qgiv
2. barnstorm
3. hobnob
4. aggregator
5. auction

In [102]:
q = "select type, id from form"
df = redshift_query_read(q, schema="production")
df.head(3)

,type,id
0,1,17
1,1,33
2,1,49


In [103]:
q = '''select 
            org, form, count(id) as trans_count, sum(amount) as trans_sum 
        from transactions_dup
        where status='A' and date>='2020-01-01' and date<'2021-01-01'
        group by form, org'''
trans_2019 = redshift_query_read(q, schema="public")

In [104]:
trans_2019['product'] = trans_2019['form'].apply(lambda x: df[df['id']==x]['type'].iloc[0])

In [105]:
trans_2019.groupby('product')[['trans_count', 'trans_sum']].sum().reset_index()

,product,trans_count,trans_sum
0,1,1766626,2.320643e+08
1,2,9061,4.873893e+05
2,3,343391,3.138221e+07
3,5,31668,3.692870e+06


In [106]:
# omit 0 transaction forms
trans_2019 = trans_2019[trans_2019['trans_sum']>0]
trans_2019.groupby('product')['org'].count()

product
1    10707
2       44
3     1949
5      254
Name: org, dtype: int64

In [107]:
for _, prod in trans_2019.groupby('product')['trans_sum'].sum().reset_index().iterrows():
    print("product {}: ${:,.2f}".format(prod['product'], prod['trans_sum']))

product 1.0: $232,064,275.70
product 2.0: $487,389.31
product 3.0: $31,382,210.53
product 5.0: $3,692,869.66


# scratch

In [90]:
# looking for duplicates
q = '''select 
            id, count(id) as trans_count, amount
        from transactions
        where 
            status='A' and date>='2020-01-01' and date<'2021-01-01'
        group by id, amount'''
trans_2019 = redshift_query_read(q, schema="public")

In [91]:
print("len all : {:,}".format(len(trans_2019)))
print("len w/ duplicates: {:,}".format(len(trans_2019[trans_2019['trans_count']>1])))
print("len w/out dup: {:,}".format(len(trans_2019[trans_2019['trans_count']==1])))
print()
print("sum all : ${:,.2f}".format(trans_2019['amount'].sum()))
print("sum duplicates: ${:,.2f}".format(trans_2019[trans_2019['trans_count']>1]['amount'].sum()))
print("sum dup: ${:,.2f}".format(trans_2019[trans_2019['trans_count']==1]['amount'].sum()))

len all : 2,150,746
len w/ duplicates: 8,041
len w/out dup: 2,142,705

sum all : $267,626,745.20
sum duplicates: $887,976.16
sum dup: $266,738,769.04


In [92]:
# looking for duplicates
q = '''select 
            id, count(id) as trans_count, amount
        from transactions_dup
        where 
            status='A' and date>='2020-01-01' and date<'2021-01-01'
        group by id, amount'''
trans_2019 = redshift_query_read(q, schema="public")

In [93]:
print("len all : {:,}".format(len(trans_2019)))
print("len w/ duplicates: {:,}".format(len(trans_2019[trans_2019['trans_count']>1])))
print("len w/out dup: {:,}".format(len(trans_2019[trans_2019['trans_count']==1])))
print()
print("sum all : ${:,.2f}".format(trans_2019['amount'].sum()))
print("sum duplicates: ${:,.2f}".format(trans_2019[trans_2019['trans_count']>1]['amount'].sum()))
print("sum dup: ${:,.2f}".format(trans_2019[trans_2019['trans_count']==1]['amount'].sum()))

len all : 2,150,746
len w/ duplicates: 0
len w/out dup: 2,150,746

sum all : $267,626,745.20
sum duplicates: $0.00
sum dup: $267,626,745.20


In [95]:
q = '''select 
            count(id)
        from transactions
        where 
            status='A' and date>='2020-01-01' and date<'2021-01-01'
        '''
print(redshift_query_read(q, schema="public"))
q = '''select 
            count(distinct(id))
        from transactions
        where 
            status='A' and date>='2020-01-01' and date<'2021-01-01'
        '''
print(redshift_query_read(q, schema="public"))
q = '''select 
            count(id)
        from transactions_dup
        where 
            status='A' and date>='2020-01-01' and date<'2021-01-01'
        '''
print(redshift_query_read(q, schema="public"))

     count
0  2167425
     count
0  2150746
     count
0  2150746
